In [14]:
import pyworld as pw
import librosa
import numpy as np
import os
import math
import torch
from pymcd.mcd import Calculate_MCD
import audio as Audio
# wav2vecSER Model
from speechbrain.inference.interfaces import foreign_class

In [27]:
ground_truth_path = "/home/johnny9696/Desktop/DATA/dailytalk_resample/data"
#model_output_path = "/mnt/hdd_storage/di_fit_tts_output/output/dailytalk"
#model_output_path = "/mnt/hdd_storage/di_fit_tts_output/output/Fitted_v19_flowv2_MPE"
model_output_path = "/mnt/hdd_storage/di_fit_tts_output/output/dailytalk"
sampling_rate = 22050
hop_length = 256
d_list = os.listdir(model_output_path)
print(d_list)

['925', '1632', '1924', '466', '1804', '2515', '1155', '905', '1391', '718', '1311', '2524', '1645', '122', '1471', '1918', '867', '1614', '3', '382', '339', '1306', '1233', '1671', '661', '79', '1440', '2309', '1360', '2382', '2529', '236', '1735', '1848', '1193', '557', '470', '1406', '1257', '1831', '440', '1535', '1942', '1363', '2408', '247', '2514', '1180', '221', '115', '1764', '2145', '2417', '1758', '1403', '172', '472', '2459', '47', '283', '362', '800', '465', '1282', '1905', '1927', '2052', '1944', '2349', '40', '782', '2263', '1622', '1514', '54', '1435', '1850', '1602', '451', '1510', '2131', '20', '119', '2489', '2266', '2438', '2015', '1865', '2149', '1542', '766', '2447', '2304', '1068', '1815', '2016', '2273', '1873', '1947', '404', '759', '2406', '622', '712', '1485', '1953', '1625', '2497', '1854', '1102', '1774', '1516', '2400', '1780', '962', '749', '1553', '1190', '295', '1368', '855', '24', '2347', '1531', '168', '896', '280', '1394', '1108', '189', '856', '1750

In [28]:
def get_pitch(wav, sampling_rate = 22050, hop_length = 256):
    pitch, t = pw.dio(
        wav.astype(np.float64),
        sampling_rate,
        frame_period=hop_length / sampling_rate * 1000,
    )
    pitch = pw.stonemask(wav.astype(np.float64), pitch, t, sampling_rate)
    return pitch

In [29]:
def get_energy(wav):
# Compute mel-scale spectrogram and energy
        STFT = Audio.stft.TacotronSTFT(
            1024,
            256,
            1024,
            80,
            22050,
            0,
            8000,
        )
        _, energy = Audio.tools.get_mel_from_wav(wav, STFT)
        return energy
        

In [18]:
count = 0
wav_rmse = 0
f0_rmse = 0
for i in d_list:
    t_output_path = os.path.join(model_output_path, i)
    t_gt_path = os.path.join(ground_truth_path, i[1:])
    w_list = os.listdir(t_output_path)
    for wav_name in w_list:
        if wav_name[-3:] !="wav":
            continue
        gt_wav, sr = librosa.load(os.path.join(t_gt_path, wav_name))
        gen_wav, sr = librosa.load(os.path.join(t_output_path, wav_name))
        gt_p = get_pitch(gt_wav)
        gen_p = get_pitch(gen_wav)
        if len(gt_p) > len(gen_p):
            gap = len(gt_p) - len(gen_p)
            gt_p = gt_p[gap//2:-gap//2]
            gen_p = gen_p[:len(gt_p)]
        else:
            gap = len(gen_p) - len(gt_p)
            gen_p = gen_p[gap//2:-gap//2]
            gt_p = gt_p[:len(gt_p)]
        try:
            gt_log = np.nan_to_num(np.log(gt_p))
            gen_log = np.nan_to_num(np.log(gen_p))
            error = abs(gt_log-gen_log)
            non_inf_counter = 0
            rmse = 0
            for data in error:
                if data != np.inf and data != -np.inf and data>= -10 and data <=10:
                    non_inf_counter += 1
                    rmse += data ** 2
            rmse = np.sqrt(rmse/non_inf_counter)
            f0_rmse += rmse
            count += 1
            if count%100 == 0:
                print(count, f0_rmse/count)
        except:
            pass

/tmp/ipykernel_94240/3240988841.py:24: RuntimeWarning: divide by zero encountered in log
  gt_log = np.nan_to_num(np.log(gt_p))
/tmp/ipykernel_94240/3240988841.py:25: RuntimeWarning: divide by zero encountered in log
  gen_log = np.nan_to_num(np.log(gen_p))


100 0.25865553722809814
200 0.27163505877259014
300 0.2693354819566689
400 0.26869025774813915
500 0.27242750002260435
600 0.2727722910198881
700 0.2698624748030164
800 0.2685931600261765
900 0.2704759382665826
1000 0.26997689959976184
1100 0.2740851436821451
1200 0.27410256845130765
1300 0.2734715120202252
1400 0.2726896280622447
1500 0.27210604424624
1600 0.2721983646085162
1700 0.2722425866012488
1800 0.27216503515425894


In [19]:
print(f0_rmse/count)

0.27358981651310077


In [20]:
count = 0
e_rmse = 0
for i in d_list:
    t_output_path = os.path.join(model_output_path, i)
    t_gt_path = os.path.join(ground_truth_path, i[1:])
    w_list = os.listdir(t_output_path)
    for wav_name in w_list:
        if wav_name[-3:] !="wav":
            continue
        gt_wav, sr = librosa.load(os.path.join(t_gt_path, wav_name))
        gen_wav, sr = librosa.load(os.path.join(t_output_path, wav_name))
        gt_p = get_energy(gt_wav)
        gen_p = get_energy(gen_wav)
        if len(gt_p) > len(gen_p):
            gap = len(gt_p) - len(gen_p)
            gt_p = gt_p[gap//2:-gap//2]
            gen_p = gen_p[:len(gt_p)]
        else:
            gap = len(gen_p) - len(gt_p)
            gen_p = gen_p[gap//2:-gap//2]
            gt_p = gt_p[:len(gt_p)]
        try:
            gt_log = np.nan_to_num(np.log(gt_p))
            gen_log = np.nan_to_num(np.log(gen_p))
            error = abs(gt_log-gen_log)
            non_inf_counter = 0
            rmse = 0
            for data in error:
                if data != np.inf and data != -np.inf and data>= -10 and data <=10:
                    non_inf_counter += 1
                    rmse += data ** 2
            rmse = np.sqrt(rmse/non_inf_counter)
            e_rmse += rmse
            count += 1
            if count%100 == 0:
                print(count, e_rmse/count)

        except:
            pass

100 2.3185399637676203
200 2.271798993704666
300 2.246603476359974
400 2.2954430475821996
500 2.2930710966871497
600 2.2944573930442163
700 2.2937272719744835
800 2.2665241327009173
900 2.2621018848761123
1000 2.2623928438029215
1100 2.2652582863789483
1200 2.2855105738931227
1300 2.2843313161903853
1400 2.2783871658314987
1500 2.2901144045854225
1600 2.3056090099830917
1700 2.3000032527488656
1800 2.290041916358195


In [21]:
print(e_rmse/count)

2.2857055624878075


In [22]:
audio_embedder = foreign_class(source="speechbrain/emotion-recognition-wav2vec2-IEMOCAP",
                                       pymodule_file="custom_interface.py", classname="CustomEncoderWav2vec2Classifier")

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
speechbrain.lobes.models.huggingface_transformers.huggingface - Wav2Vec2Model is frozen.


In [31]:
w_count = 0
wav_rmse = 0
for i in d_list:
    t_output_path = os.path.join(model_output_path, i)
    t_gt_path = os.path.join(ground_truth_path, i)
    w_list = os.listdir(t_output_path)
    for wav_name in w_list:
        if wav_name[-3:] !="wav":
            continue
        gt_wav, sr = librosa.load(os.path.join(t_gt_path, wav_name))
        gen_wav, sr = librosa.load(os.path.join(t_output_path, wav_name))
        gt_wav_e = torch.tensor(gt_wav)
        gt_wav_emb = audio_embedder.encode_batch(gt_wav_e)
        gt_wav_emb = np.array(gt_wav_emb.squeeze(0))
        gen_wav_e = torch.tensor(gen_wav)
        gen_wav_emb = audio_embedder.encode_batch(gen_wav_e)
        gen_wav_emb = np.array(gen_wav_emb.squeeze(0))
        error = abs(gt_wav_emb - gen_wav_emb)
        rmse = np.sqrt(np.mean(error**2))
        wav_rmse += rmse
        w_count += 1
        if w_count%100 == 0:
            print(w_count, wav_rmse/w_count)

100 0.7813318032026291
200 0.7770105202496052
300 0.755288302898407
400 0.7489365226402879
500 0.7561649296879769
600 0.7581666740278403
700 0.7590387346063341
800 0.7457714810129255
900 0.7501404679235485
1000 0.7519019912555813
1100 0.7541129228811373
1200 0.7518807866175969
1300 0.7586494358514364
1400 0.7618235496538026
1500 0.7629914500117302
1600 0.762681849161163
1700 0.7616467028856277
1800 0.7605984793851773


In [32]:
print(wav_rmse/w_count)

0.7613215082088403


In [33]:
mcd_count = 0
mcd = 0
mcd_toolbox = Calculate_MCD(MCD_mode="dtw")
for i in d_list:
    t_output_path = os.path.join(model_output_path, i)
    t_gt_path = os.path.join(ground_truth_path, i)
    w_list = os.listdir(t_output_path)
    for wav_name in w_list:
        if wav_name[-3:] !="wav":
            continue
        mcd_value = mcd_toolbox.calculate_mcd(os.path.join(t_gt_path, wav_name), os.path.join(t_output_path, wav_name))
        mcd += mcd_value
        mcd_count += 1
        if mcd_count % 100 == 0:
            print(mcd_count, mcd/mcd_count)

100 7.837274787418521
200 7.783389956527858
300 7.607149453781307
400 7.71399402305241
500 7.702864844713825
600 7.590505965616717
700 7.667577334033091
800 7.668980345335637
900 7.681014444443841
1000 7.692166189020467
1100 7.682098646417587
1200 7.713465964960999
1300 7.719100154049636
1400 7.7278203337330575
1500 7.734686262335614
1600 7.75893315721556
1700 7.794222626869252
1800 7.796769063685961


In [34]:
print(mcd_count, mcd/mcd_count)

1834 7.790815976385192
